In [63]:
!pip install nxviz
import pandas as pd
import numpy as np
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations as comb
from networkx.algorithms import bipartite
import nxviz as nv

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
name_basics = "name.basics.tsv"
nb_columns = ['nconst', 
              'primaryName', 
              'birthYear', 
              'deathYear', 
              'primaryProfession']

nb_df = pd.read_csv(name_basics,sep='\t',header=0,usecols=nb_columns)
nb_df = nb_df[~nb_df['birthYear'].str.contains("N", na=False)]
nb_df.dropna()
nb_df.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor"


In [65]:
title_akas = "title.akas.tsv"
ta_columns = ['titleId', 'ordering', 'title', 'region', 'language', 'types', 'attributes', 'isOriginalTitle']
convert_dict = {'titleId': str, 'ordering': str, 'title': str, 
                'region': str, 'language': str, 'types': str, 'attributes': str, 'isOriginalTitle': str}

ta_df = pd.read_csv(title_akas,sep='\t',header=0,usecols=ta_columns,dtype=convert_dict)
ta_df = ta_df[ta_df['isOriginalTitle'] == '1']
ta_df.rename(columns={'titleId':'tconst'}, inplace=True)
ta_df.dropna()
ta_df.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,tconst,ordering,title,region,language,types,attributes,isOriginalTitle
6,tt0000001,7,Carmencita,\N,\N,original,\N,1
8,tt0000002,1,Le clown et ses chiens,\N,\N,original,\N,1
21,tt0000003,6,Pauvre Pierrot,\N,\N,original,\N,1
25,tt0000004,1,Un bon bock,\N,\N,original,\N,1
34,tt0000005,11,Blacksmith Scene,\N,\N,original,\N,1


In [66]:
title_basics = "title.basics.tsv"
tb_columns = ['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'startYear', 'isAdult', 'endYear', 'runtimeMinutes', 'genres']
convert_dict = {'tconst': str, 'titleType': str, 'primaryTitle': str, 'originalTitle': str, 
                'startYear': str, 'isAdult': object,'endYear': str, 'runtimeMinutes': str, 'genres': object}

tb_df = pd.read_csv(title_basics,sep='\t',header=0,usecols=tb_columns,dtype=convert_dict)
tb_df.dropna()
tb_df = tb_df[~tb_df['genres'].str.contains('N', na=False)]
tb_df = tb_df[~tb_df['startYear'].str.contains("N", na=False)]
tb_df = tb_df[~tb_df['isAdult'].str.contains("N", na=False)]
tb_df = tb_df.dropna(subset=['isAdult'])
tb_df = tb_df[tb_df['isAdult'].astype(int) == 0]
tb_df = tb_df[tb_df['titleType'] == "movie"]
tb_df = tb_df[tb_df['startYear'].astype(int) >= 2002]
tb_df = tb_df[tb_df['startYear'].astype(int) <= 2022]
tb_df = (tb_df.drop(columns='genres').join(tb_df['genres'].str.get_dummies(sep=',')))
tb_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,Action,Adult,...,Mystery,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
11636,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,0,2019,\N,\N,1,0,...,0,0,0,0,0,0,0,0,0,0
13080,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021,\N,133,0,0,...,0,0,0,0,0,0,0,0,0,0
61097,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,\N,70,0,0,...,0,0,0,0,0,0,0,0,0,0
66314,tt0067683,movie,Workers '71: Nothing About Us Without Us,Robotnicy 1971 - Nic o nas bez nas,0,2006,\N,47,0,0,...,0,0,0,0,0,0,0,0,0,0
67645,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,\N,122,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
title_ratings = "title.ratings.tsv"
tr_columns = ['tconst', 'averageRating', 'numVotes']

tr_df = pd.read_csv(title_ratings,sep='\t',header=0,usecols=tr_columns)
tr_df = tr_df.dropna()
tr_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1924
1,tt0000002,5.8,260
2,tt0000003,6.5,1737
3,tt0000004,5.6,175
4,tt0000005,6.2,2551


In [68]:
title_principals = "drive/MyDrive/title.principals.tsv"
tp_columns = ['tconst', 'ordering', 'nconst', 'category', 'job', 'characters']

tp_df = pd.read_csv(title_principals,sep='\t',header=0,usecols=tp_columns)
tp_df = tp_df[tp_df['category'] == "actor"]
tp_df.drop('ordering', axis=1, inplace=True) 
tp_df.drop('category', axis=1, inplace=True) 
tp_df.drop('job', axis=1, inplace=True) 
tp_df.drop('characters', axis=1, inplace=True) 
tp_df = tp_df.dropna()
tp_df.head()

,tconst,nconst
11,tt0000005,nm0443482
12,tt0000005,nm0653042
16,tt0000007,nm0179163
17,tt0000007,nm0183947
21,tt0000008,nm0653028


In [69]:
# Setup relationship dataframe to construct bipartite graph
bipartite_df = pd.merge(tp_df, tb_df, how='inner',on='tconst')
bipartite_df = bipartite_df.drop(bipartite_df.columns[2:],axis = 1)
bipartite_df.head()

,tconst,nconst
0,tt0011801,nm0459029
1,tt0011801,nm0681726
2,tt0011801,nm0726256
3,tt0011801,nm0776458
4,tt0011801,nm0666006


In [70]:
# Create dataframe with amount of appearances of each index in main dataframe (how many movies each actor has participated in)

v = bipartite_df["nconst"].value_counts()
v = v.to_frame().reset_index()
v.rename(columns={'nconst' :'value', 'index' :'nconst'}, inplace=True)
v.head()

,nconst,value
0,nm0695177,47
1,nm0000821,44
2,nm0415549,43
3,nm0007123,38
4,nm0474774,38


In [139]:
# Add actor info to v dataframe

res = pd.merge(v, nb_df, how='inner',on='nconst')
res.head(50)

,nconst,value,primaryName,birthYear,deathYear,primaryProfession
0,nm0695177,47,Prakash Raj,1965,\N,"actor,producer,director"
1,nm0000821,44,Amitabh Bachchan,1942,\N,"music_artist,actor,producer"
2,nm0415549,43,Jagathy Sreekumar,1951,\N,"actor,writer,director"
3,nm0007123,38,Mammootty,1951,\N,"actor,producer,music_department"
4,nm0474774,38,Akshay Kumar,1967,\N,"actor,producer,soundtrack"
5,nm0222881,38,Tony Devon,1951,2022,"actor,producer,writer"
6,nm0222426,36,Ajay Devgn,1969,\N,"actor,producer,music_department"
7,nm0226770,35,Dileep,1968,\N,"actor,producer,assistant_director"
8,nm0482320,34,Mohanlal,1960,\N,"actor,music_department,producer"
9,nm0004109,33,Gulshan Grover,1955,\N,"actor,music_department"


In [140]:
# Filter actors with less than 6 appearances

df3 = res[res['value'] > 5]  
df3.head()

,nconst,value,primaryName,birthYear,deathYear,primaryProfession
0,nm0695177,47,Prakash Raj,1965,\N,"actor,producer,director"
1,nm0000821,44,Amitabh Bachchan,1942,\N,"music_artist,actor,producer"
2,nm0415549,43,Jagathy Sreekumar,1951,\N,"actor,writer,director"
3,nm0007123,38,Mammootty,1951,\N,"actor,producer,music_department"
4,nm0474774,38,Akshay Kumar,1967,\N,"actor,producer,soundtrack"


In [141]:
bipartite_df = pd.merge(bipartite_df, df3, how='inner',on='nconst')
bipartite_df = bipartite_df.drop(bipartite_df.columns[2:],axis = 1)
bipartite_df.head()

,tconst,nconst
0,tt0121569,nm0222881
1,tt0198794,nm0222881
2,tt0299799,nm0222881
3,tt0303327,nm0222881
4,tt0338395,nm0222881


# Create bipartite graph

In [142]:
G = nx.Graph()
G.add_nodes_from(bipartite_df['tconst'], bipartite=0)
G.add_nodes_from(bipartite_df['nconst'], bipartite=1)

In [143]:
G.add_edges_from(zip(bipartite_df['tconst'], bipartite_df['nconst']))

In [144]:
nx.is_bipartite(G)

True

In [145]:
G.number_of_nodes()

1369

In [146]:
nx.is_connected(G)

False

In [147]:
left_or_top = bipartite_df['tconst']
pos = nx.bipartite_layout(G, left_or_top)

In [148]:
actors_nodes = set(n for n,d in G.nodes(data=True) if d['bipartite']==1)
movies_nodes = set(G) - actors_nodes

In [149]:
A = bipartite.projected_graph(G, actors_nodes)

In [150]:
A.number_of_nodes()

57

In [151]:
nx.is_connected(A)

False

In [152]:
degree_centrality = nx.degree_centrality(A)
degree_centrality

{'nm0000809': 0.017857142857142856,
 'nm0943079': 0.03571428571428571,
 'nm0103977': 0.05357142857142857,
 'nm0305182': 0.0,
 'nm0045393': 0.14285714285714285,
 'nm0075710': 0.0,
 'nm0080238': 0.05357142857142857,
 'nm0712546': 0.10714285714285714,
 'nm0006763': 0.10714285714285714,
 'nm0000354': 0.017857142857142856,
 'nm0465503': 0.017857142857142856,
 'nm0000821': 0.21428571428571427,
 'nm0222881': 0.0,
 'nm0007123': 0.125,
 'nm0001016': 0.03571428571428571,
 'nm0180404': 0.017857142857142856,
 'nm0019382': 0.05357142857142857,
 'nm0261724': 0.0,
 'nm0621937': 0.08928571428571427,
 'nm0451600': 0.1607142857142857,
 'nm0415549': 0.14285714285714285,
 'nm0000579': 0.03571428571428571,
 'nm0226770': 0.10714285714285714,
 'nm0004109': 0.14285714285714285,
 'nm0000367': 0.03571428571428571,
 'nm0542343': 0.14285714285714285,
 'nm0000353': 0.03571428571428571,
 'nm0222426': 0.1607142857142857,
 'nm0482320': 0.14285714285714285,
 'nm0419688': 0.125,
 'nm0005351': 0.03571428571428571,
 'nm0

In [153]:
most_influential = list()
for node in sorted(degree_centrality, key=degree_centrality.get, reverse=True):
    most_influential.append(node)
    print(node, degree_centrality[node])

nm0000821 0.21428571428571427
nm0451600 0.1607142857142857
nm0222426 0.1607142857142857
nm0474774 0.1607142857142857
nm0792911 0.1607142857142857
nm0045393 0.14285714285714285
nm0415549 0.14285714285714285
nm0004109 0.14285714285714285
nm0542343 0.14285714285714285
nm0482320 0.14285714285714285
nm0359880 0.14285714285714285
nm0006795 0.14285714285714285
nm0007123 0.125
nm0419688 0.125
nm0007106 0.125
nm0004569 0.125
nm0329730 0.125
nm0712546 0.10714285714285714
nm0006763 0.10714285714285714
nm0226770 0.10714285714285714
nm0709359 0.10714285714285714
nm0222145 0.10714285714285714
nm0695177 0.10714285714285714
nm0621937 0.08928571428571427
nm0149822 0.08928571428571427
nm0451234 0.08928571428571427
nm0792116 0.08928571428571427
nm0103977 0.05357142857142857
nm0080238 0.05357142857142857
nm0019382 0.05357142857142857
nm0943079 0.03571428571428571
nm0001016 0.03571428571428571
nm0000579 0.03571428571428571
nm0000367 0.03571428571428571
nm0000353 0.03571428571428571
nm0005351 0.035714285714

In [154]:
mi = most_influential[:20]
mi_df = pd.DataFrame (mi, columns = ['nconst'])
mi_df.head()

,nconst
0,nm0000821
1,nm0451600
2,nm0222426
3,nm0474774
4,nm0792911


In [155]:
res = pd.merge(mi_df, nb_df, how='inner',on='nconst')
res.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession
0,nm0000821,Amitabh Bachchan,1942,\N,"music_artist,actor,producer"
1,nm0451600,Anupam Kher,1955,\N,"actor,producer,music_department"
2,nm0222426,Ajay Devgn,1969,\N,"actor,producer,music_department"
3,nm0474774,Akshay Kumar,1967,\N,"actor,producer,soundtrack"
4,nm0792911,Suniel Shetty,1961,\N,"actor,producer"


In [156]:
print(list(nx.connected_components(A)))

[{'nm0000367', 'nm0000809', 'nm0180404'}, {'nm0943079', 'nm0008346', 'nm0038355', 'nm0938893', 'nm0465503', 'nm0643885'}, {'nm0103977', 'nm0045393', 'nm0080238', 'nm0712546', 'nm0006763', 'nm0000821', 'nm0007123', 'nm0019382', 'nm0621937', 'nm0451600', 'nm0415549', 'nm0226770', 'nm0004109', 'nm0542343', 'nm0222426', 'nm0482320', 'nm0419688', 'nm0474774', 'nm0149822', 'nm0792911', 'nm0007106', 'nm0359880', 'nm0004569', 'nm0451234', 'nm0792116', 'nm0329730', 'nm0709359', 'nm0222145', 'nm0695177', 'nm0006795'}, {'nm0305182'}, {'nm0075710'}, {'nm0000997', 'nm0001016', 'nm0001426', 'nm0000151', 'nm0000354', 'nm0000514'}, {'nm0222881'}, {'nm0261724'}, {'nm0005351', 'nm0000800', 'nm0000191', 'nm0000115', 'nm0000168', 'nm0000579', 'nm0000353'}, {'nm0396069'}]


In [157]:
print(nx.number_connected_components(A))

10


In [158]:
closeness_centrality = nx.closeness_centrality(A)
closeness_centrality

{'nm0000809': 0.023809523809523808,
 'nm0943079': 0.040584415584415584,
 'nm0103977': 0.1687399678972713,
 'nm0305182': 0.0,
 'nm0045393': 0.23837868480725624,
 'nm0075710': 0.0,
 'nm0080238': 0.1687399678972713,
 'nm0712546': 0.23465401785714288,
 'nm0006763': 0.23465401785714288,
 'nm0000354': 0.02976190476190476,
 'nm0465503': 0.02976190476190476,
 'nm0000821': 0.31287202380952384,
 'nm0222881': 0.0,
 'nm0007123': 0.20023809523809524,
 'nm0001016': 0.040584415584415584,
 'nm0180404': 0.023809523809523808,
 'nm0019382': 0.1687399678972713,
 'nm0261724': 0.0,
 'nm0621937': 0.23837868480725624,
 'nm0451600': 0.25892857142857145,
 'nm0415549': 0.21454081632653066,
 'nm0000579': 0.040178571428571425,
 'nm0226770': 0.24222350230414746,
 'nm0004109': 0.25029761904761905,
 'nm0000367': 0.03571428571428571,
 'nm0542343': 0.21454081632653066,
 'nm0000353': 0.04945054945054945,
 'nm0222426': 0.27810846560846564,
 'nm0482320': 0.2634711779448622,
 'nm0419688': 0.23104395604395608,
 'nm0005351':

In [159]:
most_influential2 = list()
for node in sorted(closeness_centrality, key=closeness_centrality.get, reverse=True):
    most_influential2.append(node)
    print(node, closeness_centrality[node])

nm0000821 0.31287202380952384
nm0222426 0.27810846560846564
nm0695177 0.27810846560846564
nm0482320 0.2634711779448622
nm0451600 0.25892857142857145
nm0006795 0.2545399515738499
nm0004109 0.25029761904761905
nm0474774 0.2461943793911007
nm0226770 0.24222350230414746
nm0792911 0.24222350230414746
nm0007106 0.24222350230414746
nm0004569 0.24222350230414746
nm0045393 0.23837868480725624
nm0621937 0.23837868480725624
nm0451234 0.23837868480725624
nm0712546 0.23465401785714288
nm0006763 0.23465401785714288
nm0419688 0.23104395604395608
nm0709359 0.21765010351966874
nm0415549 0.21454081632653066
nm0542343 0.21454081632653066
nm0359880 0.21454081632653066
nm0149822 0.21151911468812878
nm0792116 0.21151911468812878
nm0007123 0.20023809523809524
nm0329730 0.20023809523809524
nm0222145 0.1900994575045208
nm0103977 0.1687399678972713
nm0080238 0.1687399678972713
nm0019382 0.1687399678972713
nm0005351 0.05357142857142857
nm0001426 0.04960317460317461
nm0938893 0.04960317460317461
nm0000514 0.04960

In [160]:
mi2 = most_influential2[:20]
mi2_df = pd.DataFrame (mi2, columns = ['nconst'])
mi2_df.head()
res2 = pd.merge(mi2_df, nb_df, how='inner',on='nconst')
res2.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession
0,nm0000821,Amitabh Bachchan,1942,\N,"music_artist,actor,producer"
1,nm0222426,Ajay Devgn,1969,\N,"actor,producer,music_department"
2,nm0695177,Prakash Raj,1965,\N,"actor,producer,director"
3,nm0482320,Mohanlal,1960,\N,"actor,music_department,producer"
4,nm0451600,Anupam Kher,1955,\N,"actor,producer,music_department"


# Create 

In [161]:
movies_nodes_2 = set(n for n,d in G.nodes(data=True) if d['bipartite']==0)
actors_nodes_2 = set(G) - movies_nodes_2

In [162]:
A2 = bipartite.projected_graph(G, movies_nodes_2)

In [163]:
nx.is_connected(A2)

False

In [164]:
A2.number_of_nodes()

1312

In [165]:
degree_centrality2 = nx.degree_centrality(A2)
degree_centrality2

{'tt0266543': 0.016781083142639208,
 'tt0407297': 0.01906941266209001,
 'tt0990363': 0.016018306636155607,
 'tt0314786': 0.015255530129672007,
 'tt0800981': 0.022883295194508012,
 'tt0810988': 0.015255530129672007,
 'tt0337633': 0.038901601830663615,
 'tt0198794': 0.028222730739893214,
 'tt0402580': 0.01754385964912281,
 'tt0497640': 0.016781083142639208,
 'tt0337943': 0.015255530129672007,
 'tt0988655': 0.016781083142639208,
 'tt0887764': 0.016781083142639208,
 'tt0360797': 0.018306636155606407,
 'tt0922642': 0.02212051868802441,
 'tt1102159': 0.038901601830663615,
 'tt0347330': 0.018306636155606407,
 'tt0435390': 0.02440884820747521,
 'tt0455195': 0.01906941266209001,
 'tt0380070': 0.01906941266209001,
 'tt0475263': 0.01906941266209001,
 'tt0462465': 0.01906941266209001,
 'tt0444915': 0.04729214340198322,
 'tt0357340': 0.048054919908466824,
 'tt0814285': 0.016781083142639208,
 'tt0839969': 0.015255530129672007,
 'tt0244521': 0.015255530129672007,
 'tt0790736': 0.015255530129672007,
 

In [166]:
degree_centrality2 == degree_centrality

False

In [172]:
closeness_centrality2 = nx.closeness_centrality(A2)
closeness_centrality2 == closeness_centrality

False

In [175]:
eigenvector_centrality2 = nx.eigenvector_centrality(A2, max_iter=1000)


In [177]:
betweenness_centrality2 = nx.betweenness_centrality(A2)

In [169]:
most_influential_movies = list()
for node in sorted(degree_centrality2, key=degree_centrality2.get, reverse=True):
    most_influential_movies.append(node)
    print(node, degree_centrality2[node])
print(most_influential_movies[:20])

tt1138482 0.08848207475209764
tt0347332 0.08161708619374523
tt0473310 0.08161708619374523
tt0320097 0.07398932112890923
tt0476550 0.07322654462242563
tt0924213 0.07322654462242563
tt0330082 0.07093821510297484
tt0811066 0.07017543859649124
tt0456481 0.07017543859649124
tt0294662 0.07017543859649124
tt0413366 0.06788710907704043
tt0355729 0.06712433257055683
tt0382733 0.06559877955758962
tt0419058 0.06483600305110603
tt0363409 0.06483600305110603
tt0319020 0.06483600305110603
tt0444913 0.06331045003813883
tt0347416 0.06331045003813883
tt0463650 0.06102212051868803
tt0406622 0.060259344012204424
tt0415768 0.059496567505720827
tt0410591 0.059496567505720827
tt0463644 0.05873379099923723
tt0393724 0.05873379099923723
tt0808500 0.05873379099923723
tt0397882 0.057971014492753624
tt0306434 0.057971014492753624
tt0390614 0.057971014492753624
tt0437182 0.057971014492753624
tt0338721 0.057208237986270026
tt11460992 0.057208237986270026
tt10623274 0.05644546147978643
tt0346172 0.05644546147978643

In [170]:
mim = most_influential_movies[:20]
mim_df = pd.DataFrame (mim, columns = ['tconst'])
mim_df.head()

,tconst
0,tt1138482
1,tt0347332
2,tt0473310
3,tt0320097
4,tt0476550


# Degree

In [171]:
# 
res = pd.merge(mim_df, tb_df, how='inner',on='tconst')
res.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,Action,Adult,...,Mystery,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,tt1138482,movie,Twenty:20,Twenty:20,0,2008,\N,195,1,0,...,0,0,0,0,0,0,0,0,0,0
1,tt0347332,movie,Khakee,Khakee,0,2004,\N,174,1,0,...,0,0,0,0,0,0,0,0,0,0
2,tt0473310,movie,Ram Gopal Varma Ki Aag,Ram Gopal Varma Ki Aag,0,2007,\N,171,1,0,...,0,0,0,0,0,0,0,0,0,0
3,tt0320097,movie,Hum Kisi Se Kum Nahin,Hum Kisi Se Kum Nahin,0,2002,\N,161,1,0,...,0,0,0,0,0,0,0,0,0,0
4,tt0476550,movie,God Tussi Great Ho,God Tussi Great Ho,0,2008,\N,152,0,0,...,0,0,0,0,0,0,0,0,0,0


# Closeness

In [178]:
most_influential_movies = list()
for node in sorted(closeness_centrality2, key=closeness_centrality2.get, reverse=True):
    most_influential_movies.append(node)

In [179]:
mim = most_influential_movies[:20]
mim_df = pd.DataFrame (mim, columns = ['tconst'])
mim_df.head()

,tconst
0,tt0473310
1,tt0476550
2,tt0346172
3,tt0824375
4,tt0320097


In [180]:
res = pd.merge(mim_df, tb_df, how='inner',on='tconst')
res.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,Action,Adult,...,Mystery,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,tt0473310,movie,Ram Gopal Varma Ki Aag,Ram Gopal Varma Ki Aag,0,2007,\N,171,1,0,...,0,0,0,0,0,0,0,0,0,0
1,tt0476550,movie,God Tussi Great Ho,God Tussi Great Ho,0,2008,\N,152,0,0,...,0,0,0,0,0,0,0,0,0,0
2,tt0346172,movie,Zamaanat: And Justice for All,Zamaanat: And Justice for All,0,2021,\N,\N,0,0,...,1,0,0,0,0,0,0,0,0,0
3,tt0824375,movie,Nishabd,Nishabd,0,2007,\N,110,0,0,...,0,0,0,0,0,0,0,0,0,0
4,tt0320097,movie,Hum Kisi Se Kum Nahin,Hum Kisi Se Kum Nahin,0,2002,\N,161,1,0,...,0,0,0,0,0,0,0,0,0,0


# Eigenvector

In [193]:
most_influential_movies = list()
for node in sorted(eigenvector_centrality2, key=eigenvector_centrality2.get, reverse=True):
    most_influential_movies.append(node)

In [194]:
mim = most_influential_movies[:20]
mim_df = pd.DataFrame (mim, columns = ['tconst'])
mim_df.head()

,tconst
0,tt0347332
1,tt0320097
2,tt0473310
3,tt0811066
4,tt0294662


In [195]:
res = pd.merge(mim_df, tb_df, how='inner',on='tconst')
res.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,Action,Adult,...,Mystery,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,tt0347332,movie,Khakee,Khakee,0,2004,\N,174,1,0,...,0,0,0,0,0,0,0,0,0,0
1,tt0320097,movie,Hum Kisi Se Kum Nahin,Hum Kisi Se Kum Nahin,0,2002,\N,161,1,0,...,0,0,0,0,0,0,0,0,0,0
2,tt0473310,movie,Ram Gopal Varma Ki Aag,Ram Gopal Varma Ki Aag,0,2007,\N,171,1,0,...,0,0,0,0,0,0,0,0,0,0
3,tt0811066,movie,Shootout at Lokhandwala,Shootout at Lokhandwala,0,2007,\N,122,1,0,...,0,0,0,0,0,0,0,0,0,0
4,tt0294662,movie,Kaante,Kaante,0,2002,\N,154,1,0,...,0,0,0,0,0,0,0,0,0,0


# Betweenness

In [190]:
most_influential_movies = list()
for node in sorted(betweenness_centrality2, key=betweenness_centrality2.get, reverse=True):
    most_influential_movies.append(node)

In [191]:
mim = most_influential_movies[:20]
mim_df = pd.DataFrame (mim, columns = ['tconst'])
mim_df.head()

,tconst
0,tt0473310
1,tt0824375
2,tt1138482
3,tt10940482
4,tt0816605


In [192]:
res = pd.merge(mim_df, tb_df, how='inner',on='tconst')
res.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,Action,Adult,...,Mystery,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,tt0473310,movie,Ram Gopal Varma Ki Aag,Ram Gopal Varma Ki Aag,0,2007,\N,171,1,0,...,0,0,0,0,0,0,0,0,0,0
1,tt0824375,movie,Nishabd,Nishabd,0,2007,\N,110,0,0,...,0,0,0,0,0,0,0,0,0,0
2,tt1138482,movie,Twenty:20,Twenty:20,0,2008,\N,195,1,0,...,0,0,0,0,0,0,0,0,0,0
3,tt10940482,movie,Oru Avadhikaalam,Oru Avadhikaalam,0,2010,\N,85,0,0,...,0,0,0,0,0,0,0,0,0,0
4,tt0816605,movie,Paramasivan,Paramasivan,0,2006,\N,160,1,0,...,0,0,0,0,0,0,0,1,0,0
